#### How to use this notebook?

```
conda activate deepFCD
pip install jupyterlab
git clone https://github.com/NOEL-MNI/deepFCD
cd deepFCD/app
```

- Using the GPU: cuda0
    ```
    THEANO_FLAGS=mode=FAST_RUN,device=cuda0,floatX=float32,dnn.enabled=False jupyter lab --no-browser .
    ```

- Using the GPU: cudaX [replace X with the desired GPU-ID]
    ```
    THEANO_FLAGS=mode=FAST_RUN,device=cudaX,floatX=float32,dnn.enabled=False jupyter lab --no-browser .
    ```

- Using the CPU [not recommended, runtimes can exceed several hours/days depending on the hardware]
    ```
    THEANO_FLAGS=mode=FAST_RUN,device=cpu,floatX=float32,dnn.enabled=False jupyter lab --no-browser .
    ```

#### resources:
- https://www.tensorflow.org/guide/keras/transfer_learning
- https://learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/
- https://stackoverflow.com/questions/41668813/how-to-add-and-remove-new-layers-in-keras-after-loading-weights
- https://stackoverflow.com/questions/61550788/is-there-a-way-to-freeze-specific-layers-in-a-keraslayer

In [39]:
import os
import sys
import multiprocessing
from config.experiment import options
import warnings
warnings.filterwarnings('ignore')
import time
import numpy as np
import setproctitle as spt
from tqdm import tqdm

os.environ["KERAS_BACKEND"] = "theano"

# deepFCD imports
from models.noel_models_keras import *
from keras.models import load_model, Model
from keras.utils.layer_utils import count_params
from keras import backend as K
from keras.optimizers import Adadelta, Adam
from keras import losses
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, LambdaCallback
from keras.utils.np_utils import to_categorical
from keras.utils.io_utils import HDF5Matrix
from utils.metrics import *
from utils.base import *

# deepMask imports
import torch
from mo_dots import Data
from deepMask.app.utils.data import *
from deepMask.app.utils.deepmask import *
from deepMask.app.utils.image_processing import noelImageProcessor
import deepMask.app.vnet as vnet

### Directory Organization for Training Data
```bash
/data
├── brain/ 
│   ├── FCD_001_1_flair.nii.gz
│   ├── FCD_001_1_t1.nii.gz
│   ├── FCD_002_1_flair.nii.gz
│   ├── FCD_002_1_t1.nii.gz
│   ├── FCD_003_1_flair.nii.gz
│   ├── FCD_003_1_t1.nii.gz
│   ├── FCD_004_1_flair.nii.gz
│   ├── FCD_004_1_t1.nii.gz
│   ├── FCD_005_1_flair.nii.gz
│   ├── FCD_005_1_t1.nii.gz
│   ├── FCD_006_1_flair.nii.gz
│   └── FCD_006_1_t1.nii.gz
│
├── lesion_labels/
│   ├── FCD_001_1_lesion.nii.gz
│   ├── FCD_002_1_lesion.nii.gz
│   ├── FCD_003_1_lesion.nii.gz
│   ├── FCD_004_1_lesion.nii.gz
│   ├── FCD_005_1_lesion.nii.gz
│   └── FCD_006_1_lesion.nii.gz
│
├── hdf5/
│   └── exp_dropoutMC_FCD_data.h5
│   
└── noel_deepFCD_dropoutMC  # [deepFCD output images for a single patient listed]
    ├── FCD_001_noel_deepFCD_dropoutMC_prob_mean_0.nii.gz # [mean PROBABILITY image from CNN-1]
    ├── FCD_001_noel_deepFCD_dropoutMC_prob_mean_1.nii.gz # [mean PROBABILITY image from CNN-2]
    ├── FCD_001_noel_deepFCD_dropoutMC_prob_var_0.nii.gz  # [mean UNCERTAINTY image from CNN-1]
    └── FCD_001_noel_deepFCD_dropoutMC_prob_var_1.nii.gz  # [mean UNCERTAINTY image from CNN-2]
```

#### Note: The below configuration assumes T1 and T2 contrasts are in their native sterotaxic space, and therefore, will be preprocessed before brain extraction and FCD detection.
#### Preprocessing entails: 1) T1 and T1 co-registration to MNI template space, and 2) Intensity non-uniformity correction

### Configuration

In [53]:
# global configuration
args = Data()
args.dir = '/tmp/deepFCD'
args.brain_masking = False # set to True or any non-zero value for brain extraction or skull-removal, False otherwise
args.preprocess = True # co-register T1 and T2 contrasts before brain extraction
args.seed = 666

args.train = ['mcd_131_1', 'mcd_132_1', 'mcd_133_1', 'mcd_134_1']
args.test = ['mcd_135_1', 'mcd_136_1']
cwd = os.getcwd()

# deepFCD configuration
K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')  # TH dimension ordering in this code

options['parallel_gpu'] = False
modalities = ['T1', 'FLAIR']
x_names = options['x_names']
options["y_names"] = ["_lesion.nii.gz"]
y_names = options["y_names"]
options['dropout_mc'] = True
options['batch_size'] = 350000
options['mini_batch_size'] = 2048
options['load_checkpoint_1'] = True
options['load_checkpoint_2'] = True

# trained model weights based on 148 histologically-verified FCD subjects
options['test_folder'] = args.dir
options['weight_paths'] = os.path.join(cwd, 'weights')
options['experiment'] = 'noel_deepFCD_dropoutMC'
spt.setproctitle(options['experiment'])
print("experiment: {}".format(options['experiment']))




experiment: noel_deepFCD_dropoutMC


### Initialize the CNN, and load the model weights from disk

In [7]:
# initialize empty model
model = None
# initialize the CNN architecture
model = off_the_shelf_model(options)

load_weights = os.path.join(options['weight_paths'], 'noel_deepFCD_dropoutMC_model_1.h5')
print("loading DNN1, model[0]: {} exists".format(load_weights)) if os.path.isfile(load_weights) else sys.exit("model[0]: {} doesn't exist".format(load_weights))
model[0] = load_model(load_weights)

load_weights = os.path.join(options['weight_paths'], 'noel_deepFCD_dropoutMC_model_2.h5')
print("loading DNN2, model[1]: {} exists".format(load_weights)) if os.path.isfile(load_weights) else sys.exit("model[1]: {} doesn't exist".format(load_weights))
model[1] = load_model(load_weights)
print(model[1].summary())

loading DNN1, model[0]: /local_raid/data/ravnoor/02_docker/deepFCD/app/weights/noel_deepFCD_dropoutMC_model_1.h5 exists
loading DNN2, model[1]: /local_raid/data/ravnoor/02_docker/deepFCD/app/weights/noel_deepFCD_dropoutMC_model_2.h5 exists
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2, 16, 16, 16)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 48, 16, 16, 16)    2640      
_________________________________________________________________
batch_normalization_3 (Batch (None, 48, 16, 16, 16)    192       
_________________________________________________________________
dropout_4 (Dropout)          (None, 48, 16, 16, 16)    0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 48, 8, 8, 8)       0         
__________________________________

In [8]:
print("trainable layers:", model[0].trainable_weights)


trainable layers: [conv3d_1/kernel, conv3d_1/bias, batch_normalization_1/gamma, batch_normalization_1/beta, conv3d_2/kernel, conv3d_2/bias, batch_normalization_2/gamma, batch_normalization_2/beta, conv3d_3/kernel, conv3d_3/bias]


### Freeze the weights of selected layers
#### only 3/5 possible trainable layers for `model[1]` are frozen
You could do the same for `model[0]`

#### Caveats for freezing the `batch_normalization` layers - [source](https://www.tensorflow.org/guide/keras/transfer_learning)
`BatchNormalization` contains 2 non-trainable weights that get updated during training. These are the variables tracking the mean and variance of the inputs.

When you set `bn_layer.trainable = False`, the `BatchNormalization` layer will run in inference mode, and will not update its mean & variance statistics. This is not the case for other layers in general, as [weight trainability & inference/training modes are two orthogonal concepts](https://keras.io/getting_started/faq/#whats-the-difference-between-the-training-argument-in-call-and-the-trainable-attribute). But the two are tied in the case of the `BatchNormalization` layer.

When you unfreeze a model that contains `BatchNormalization` layers in order to do fine-tuning, you should keep the `BatchNormalization` layers in inference mode by passing training=False when calling the base model. Otherwise the updates applied to the non-trainable weights will suddenly destroy what the model has learned.


In [9]:
trainable_layers = ['conv3d_1', 'batch_normalization_1', 'conv3d_2', 'batch_normalization_2' 'conv3d_3']
freeze_layers = ['conv3d_1', 'conv3d_2']

model[1].trainable = True
for layer in model[0].layers:
    # selecting layer by name
    for frozen in freeze_layers:
        if layer.name == frozen:
            print(frozen)
            layer.trainable = False          
            

conv3d_1
batch_normalization_1
conv3d_2


In [10]:
# Check the trainable status of the individual layers
for layer in model[0].layers:
    print('{:>25} {:}'.format(layer.name, layer.trainable))
    

                  input_1 False
                 conv3d_1 False
    batch_normalization_1 False
                dropout_1 True
          max_pooling3d_1 True
                 conv3d_2 False
    batch_normalization_2 True
                dropout_2 True
          max_pooling3d_2 True
                dropout_3 True
                 conv3d_3 True
          max_pooling3d_3 True
                flatten_1 True
             activation_1 True


In [11]:
trainable_count = count_params(model[0].trainable_weights)
non_trainable_count = count_params(model[0].non_trainable_weights)
print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))


Total params: 132,914
Trainable params: 5,378
Non-trainable params: 127,536


### Remove all the layers after `conv3d_3` layer, and add back the `MaxPooling3D()`, `Flatten()`, and `Activation()` layers
#### This isn't entriely necessary, but gives you the flexibility to add your own custom or standard layers

In [12]:
layer_name = 'conv3d_3'
# base_model = Model(inputs=model[0].input, outputs=model[0].get_layer(layer_name).output)
x = MaxPooling3D((4, 4, 4))(model[0].get_layer(layer_name).output)
x = Flatten()(x)
# Output
out = Activation('softmax')(x)
new_model = Model(inputs=model[0].input, output=[out])

In [13]:
print(new_model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2, 16, 16, 16)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 48, 16, 16, 16)    2640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 16, 16, 16)    192       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 16, 16, 16)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 48, 8, 8, 8)       0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 96, 8, 8, 8)       124512    
_________________________________________________________________
batch_normalization_2 (Batch (None, 96, 8, 8, 8)       384       
__________

In [14]:
new_model.compile(optimizer=Adadelta(), loss=losses.binary_crossentropy, metrics=['accuracy'])

#### Specify your own data to use to train the partially frozen model

In [63]:
options["main_dir"] = args.dir
options["model_dir"] = os.path.join(options["main_dir"], "models")
# save the sampled patches as a HDF5 dataset for faster subsequent experiments
options["save_as_hdf5"] = True 
options["hdf5_data_dir"] = os.path.join(options["main_dir"], "data", "hdf5")

options["compute_performance"] = False
sensitivity = 0
perf = {}

options["train_folder"] = os.path.join(options["main_dir"], "data")
options["test_folder"] = os.path.join(options["main_dir"], "data")

In [21]:
train = args.train

options["load_checkpoint_1"] = False
options["load_checkpoint_2"] = False
# options['continue_training_2'] = True
# options['initial_epoch_2'] = 69

train_list = []
train_data = {}

for i in train:
    train_list.append(i)

train_data = {
    f: {
        m: os.path.join(options["train_folder"], "brain", f + n)
        for m, n in zip(modalities, x_names)
    }
    for f in train_list
}
train_labels = {
    f: os.path.join(options["train_folder"], "lesion_labels", f + y_names[0])
    for f in train_list
}

In [23]:
train_data, train_labels

({'mcd_131_1': {'T1': '/tmp/deepFCD/data/brain/mcd_131_1_t1.nii.gz',
   'FLAIR': '/tmp/deepFCD/data/brain/mcd_131_1_flair.nii.gz'},
  'mcd_132_1': {'T1': '/tmp/deepFCD/data/brain/mcd_132_1_t1.nii.gz',
   'FLAIR': '/tmp/deepFCD/data/brain/mcd_132_1_flair.nii.gz'},
  'mcd_133_1': {'T1': '/tmp/deepFCD/data/brain/mcd_133_1_t1.nii.gz',
   'FLAIR': '/tmp/deepFCD/data/brain/mcd_133_1_flair.nii.gz'},
  'mcd_134_1': {'T1': '/tmp/deepFCD/data/brain/mcd_134_1_t1.nii.gz',
   'FLAIR': '/tmp/deepFCD/data/brain/mcd_134_1_flair.nii.gz'},
  'mcd_135_1': {'T1': '/tmp/deepFCD/data/brain/mcd_135_1_t1.nii.gz',
   'FLAIR': '/tmp/deepFCD/data/brain/mcd_135_1_flair.nii.gz'}},
 {'mcd_131_1': '/tmp/deepFCD/data/lesion_labels/mcd_131_1_lesion.nii.gz',
  'mcd_132_1': '/tmp/deepFCD/data/lesion_labels/mcd_132_1_lesion.nii.gz',
  'mcd_133_1': '/tmp/deepFCD/data/lesion_labels/mcd_133_1_lesion.nii.gz',
  'mcd_134_1': '/tmp/deepFCD/data/lesion_labels/mcd_134_1_lesion.nii.gz',
  'mcd_135_1': '/tmp/deepFCD/data/lesion_la

### Load data from individual NIfTI images and extract patches

In [24]:
X, labels = load_training_data(train_data, train_labels, options, subcort_masks=None)
y = to_categorical(labels, num_classes=2)

print( '====> # 3D training patches:', X.shape[0] ,'\n' )
print( '====> # patch size:', (X.shape[2],X.shape[3],X.shape[4]) ,'\n' )
print( '====> # modalities:', (X.shape[1]) ,'\n' )

====> # 3D training patches: 34002 

====> # patch size: (16, 16, 16) 

====> # modalities: 2 



In [28]:
options['mini_batch_size'] = 2048
batch_size = options['mini_batch_size']//2

net_model = 'model_1'
options['weight_paths'] = options["model_dir"]
net_weights = os.path.join(options['weight_paths'], 'checkpoints') + '/' + net_model + '_weights.h5'

net_logs = os.path.join(options['weight_paths'], 'logs')
if not os.path.exists(net_logs):
    os.mkdir(os.path.join(options['weight_paths'], 'checkpoints'))
    os.mkdir(net_logs)

RAND = time.strftime('%a''_' '%H_%M_%S')
early_stopping_monitor, model_checkpoint, csv_logger, json_logging_callback = model_callbacks(net_weights, net_model, net_logs, options, RAND)

In [37]:
print( '====> DNN1 // fitting model', '\n' )
options['max_epochs_1'] = options['max_epochs_1']//2
new_model.fit(
            X, y, batch_size=batch_size, epochs=options['max_epochs_1'],
            verbose=2, shuffle="batch", validation_split=options['train_split'], # validation_data=(X_val, y_val),
            callbacks=[early_stopping_monitor, model_checkpoint]
            )

====> DNN1 // fitting model 

Train on 25501 samples, validate on 8501 samples
Epoch 1/30
 - 4s - loss: 3.4262e-04 - acc: 0.9999 - val_loss: 6.9768e-05 - val_acc: 1.0000
Epoch 2/30
 - 4s - loss: 8.1720e-04 - acc: 0.9997 - val_loss: 9.8393e-05 - val_acc: 1.0000
Epoch 3/30
 - 4s - loss: 2.8628e-04 - acc: 1.0000 - val_loss: 1.1803e-04 - val_acc: 1.0000
Epoch 4/30
 - 4s - loss: 4.3838e-04 - acc: 0.9998 - val_loss: 1.5212e-04 - val_acc: 0.9999
Epoch 5/30
 - 4s - loss: 3.4361e-04 - acc: 0.9999 - val_loss: 1.4174e-04 - val_acc: 1.0000
Epoch 6/30
 - 4s - loss: 3.9617e-04 - acc: 0.9998 - val_loss: 6.1269e-05 - val_acc: 1.0000
Epoch 7/30
 - 4s - loss: 3.3724e-04 - acc: 0.9998 - val_loss: 6.0597e-05 - val_acc: 1.0000
Epoch 8/30
 - 4s - loss: 3.3114e-04 - acc: 0.9998 - val_loss: 1.3225e-04 - val_acc: 1.0000
Epoch 9/30
 - 4s - loss: 3.9629e-04 - acc: 0.9998 - val_loss: 2.1273e-04 - val_acc: 0.9999
Epoch 10/30
 - 4s - loss: 3.9247e-04 - acc: 0.9999 - val_loss: 2.9889e-04 - val_acc: 0.9999
Epoch 11/3

### Fine-tune the model with all layers "unfrozen", with a much smaller learning rate

In [51]:
# Unfreeze the base model
# new_model.trainable = True   
for layer in new_model.layers:
    # selecting layer by name
    if layer.name.startswith('batch_normalization'):
        layer.trainable = False
    else:
        layer.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
new_model.compile(optimizer=Adam(1e-5), loss=losses.binary_crossentropy, metrics=['accuracy'])

# Check the trainable status of the individual layers
for layer in new_model.layers:
    print('{:>25} {:}'.format(layer.name, layer.trainable))

                  input_1 True
                 conv3d_1 True
    batch_normalization_1 False
                dropout_1 True
          max_pooling3d_1 True
                 conv3d_2 True
    batch_normalization_2 False
                dropout_2 True
          max_pooling3d_2 True
                dropout_3 True
                 conv3d_3 True
          max_pooling3d_7 True
                flatten_3 True
             activation_3 True


In [50]:
# Train end-to-end using your new dataset
print( '====> DNN1 // fine-tuning the model', '\n')
options['max_epochs_1'] = options['max_epochs_1']//2
new_model.fit(
            X, y, batch_size=batch_size, epochs=options['max_epochs_1'],
            verbose=2, shuffle="batch", validation_split=options['train_split'],
            callbacks=[early_stopping_monitor, model_checkpoint]
            )

====> DNN1 // fine-tuning the model 

Train on 25501 samples, validate on 8501 samples
Epoch 1/30
 - 7s - loss: 1.8968e-04 - acc: 1.0000 - val_loss: 7.5791e-05 - val_acc: 1.0000
Epoch 2/30
 - 7s - loss: 2.8420e-04 - acc: 0.9999 - val_loss: 7.7897e-05 - val_acc: 1.0000
Epoch 3/30
 - 7s - loss: 3.2759e-04 - acc: 0.9998 - val_loss: 7.7422e-05 - val_acc: 1.0000
Epoch 4/30
 - 7s - loss: 1.4612e-04 - acc: 1.0000 - val_loss: 7.5621e-05 - val_acc: 1.0000
Epoch 5/30
 - 7s - loss: 8.4724e-05 - acc: 1.0000 - val_loss: 7.5989e-05 - val_acc: 1.0000
Epoch 6/30
 - 7s - loss: 1.5421e-04 - acc: 1.0000 - val_loss: 7.7371e-05 - val_acc: 1.0000
Epoch 7/30
 - 7s - loss: 3.9578e-04 - acc: 0.9998 - val_loss: 7.7833e-05 - val_acc: 1.0000
Epoch 8/30
 - 7s - loss: 1.4936e-04 - acc: 1.0000 - val_loss: 8.1342e-05 - val_acc: 1.0000
Epoch 9/30
 - 7s - loss: 1.2544e-04 - acc: 1.0000 - val_loss: 8.5506e-05 - val_acc: 1.0000
Epoch 10/30
 - 7s - loss: 1.8176e-04 - acc: 1.0000 - val_loss: 8.6907e-05 - val_acc: 1.0000
Ep

### Load the test data and get predictions from fine-tuned CNN model

In [64]:
test = args.test

test_list = []
test_data = {}

for i in test:
    test_list.append(i)

test_data = {
    f: {
        m: os.path.join(options["test_folder"], "brain", f + n)
        for m, n in zip(modalities, x_names)
    }
    for f in test_list
}
test_labels = {
    f: os.path.join(options["test_folder"], "lesion_labels", f + y_names[0])
    for f in test_list
}

In [65]:
test_data, test_list

({'mcd_135_1': {'T1': '/tmp/deepFCD/data/brain/mcd_135_1_t1.nii.gz',
   'FLAIR': '/tmp/deepFCD/data/brain/mcd_135_1_flair.nii.gz'},
  'mcd_136_1': {'T1': '/tmp/deepFCD/data/brain/mcd_136_1_t1.nii.gz',
   'FLAIR': '/tmp/deepFCD/data/brain/mcd_136_1_flair.nii.gz'}},
 ['mcd_135_1', 'mcd_136_1'])

In [66]:
# model_pair is a list of models to be used for inference
# here we replace model[0] with our new_model
# in its vannila form, old_model = [model[0] model[1]]
new_models = [new_model, model[1]]

for _, scan in enumerate(tqdm(test_list, desc='serving predictions using the trained model', colour='blue')):
    t_data = {}
    t_data[scan] = test_data[scan]

    options['pred_folder'] = os.path.join(options['test_folder'], scan, options['experiment'])
    if not os.path.exists(options['pred_folder']):
        os.makedirs(options['pred_folder'])

    pred_mean_fname = os.path.join(options['pred_folder'], scan + '_prob_mean_1.nii.gz')
    pred_var_fname = os.path.join(options['pred_folder'], scan + '_prob_var_1.nii.gz')

    if np.logical_and(os.path.isfile(pred_mean_fname), os.path.isfile(pred_var_fname)):
        print("prediction for {} already exists".format(scan))
        continue

    options['test_scan'] = scan

    start = time.time()
    print('\n')
    print('-'*70)
    print("testing the model for scan: {}".format(scan))
    print('-'*70)

    test_model(new_models, t_data, options)

    end = time.time()
    diff = (end - start) // 60
    print("-"*70)
    print("time elapsed: ~ {} minutes".format(diff))
    print("-"*70)

serving predictions using the trained model:   0%|                                                                                                     | 0/2 [00:00<?, ?it/s]



----------------------------------------------------------------------
testing the model for scan: mcd_135_1
----------------------------------------------------------------------



serving predictions using the trained model:   0%|                                                                                                     | 0/2 [04:33<?, ?it/s]


KeyboardInterrupt: 